In [4]:
# Bibliotecas

import pandas as pd
import os
from sqlalchemy import create_engine, inspect, text, types
from io import BytesIO
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
from utils import sql_to_dbml

ImportError: cannot import name 'create_engine' from 'sqlalchemy' (unknown location)

In [9]:
# Dicionários e variáveis
db_path_silver = "../database/silver/01_silver.db"
db_path_gold = "../database/gold/02_gold.db"

## Criação das tabelas

In [10]:
# Cria conexão com banco de dados SQLite
engine_gold = create_engine(f"sqlite:///{db_path_gold}")
engine_silver = create_engine(f"sqlite:///{db_path_silver}")

In [11]:
# Criação das tabelas
create_scripts = [
"""
-- Visão mensal consolidada de lucro e margem (TOTAL)
CREATE TABLE IF NOT EXISTS g_fato_financas_mensal (
  id_data INTEGER NOT NULL, -- Referência à data representando o mês na dim_tempo
  receita_total FLOAT,     -- Receita somada de todas as fontes no mês
  despesas_total FLOAT,    -- Total de despesas no mês
  lucro FLOAT,            -- Lucro no mês (receita - despesas)
  margem_lucro FLOAT,     -- Margem de lucro no mês (lucro / receita)
  evolucao_lucro FLOAT    -- Comparação percentual com o mês anterior (lucro atual / lucro anterior - 1)
);
""",
"""
-- Visão mensal consolidada de lucro e margem (ALUCAR)
CREATE TABLE IF NOT EXISTS g_fato_financas_mensal_alucar (
  id_data INTEGER NOT NULL, -- Referência à data representando o mês na dim_tempo
  receita_total FLOAT,     -- Total de receita gerada pelas vendas da Alucar no mês
  despesas_total FLOAT,    -- Despesas atribuídas à operação da Alucar no mês
  lucro FLOAT,            -- Lucro mensal da Alucar (receita - despesas)
  margem_lucro FLOAT,     -- Margem de lucro da Alucar (lucro / receita)
  evolucao_lucro FLOAT    -- Comparação com o mês anterior (lucro atual / lucro anterior - 1)
);
""",
"""
-- Visão mensal consolidada de lucro e margem (CONSIGCAR)
CREATE TABLE IF NOT EXISTS g_fato_financas_mensal_consigcar (
  id_data INTEGER NOT NULL, -- Referência à data representando o mês na dim_tempo
  receita_total FLOAT,     -- Total de receita gerada pelas vendas da Consigcar no mês
  despesas_total FLOAT,    -- Despesas atribuídas à operação da Consigcar no mês
  lucro FLOAT,            -- Lucro mensal da Consigcar (receita - despesas)
  margem_lucro FLOAT,     -- Margem de lucro da Consigcar (lucro / receita)
  evolucao_lucro FLOAT    -- Comparação com o mês anterior (lucro atual / lucro anterior - 1)
);
""",
"""
-- Visão anual consolidada da empresa como um todo (Alucar + Consigcar)
CREATE TABLE IF NOT EXISTS g_fato_financas_anual (
  id_data INTEGER NOT NULL, -- Referência a uma data do ano na dim_tempo (pode é 01/jan)
  receita_total FLOAT,     -- Receita total combinada de todas as operações no ano
  despesas_total FLOAT,    -- Despesas totais combinadas no ano
  lucro FLOAT,            -- Lucro consolidado no ano (receita - despesas)
  margem_lucro FLOAT,     -- Margem de lucro anual consolidada
  evolucao_lucro FLOAT    -- Comparação com o ano anterior (lucro atual / lucro anterior - 1)
);
""",
"""
-- Visão anual da operação da empresa Alucar
CREATE TABLE IF NOT EXISTS g_fato_financas_anual_alucar (
  id_data INTEGER NOT NULL, -- Referência a uma data do ano na dim_tempo
  receita_total FLOAT,     -- Receita total gerada pela Alucar no ano
  despesas_total FLOAT,    -- Despesas totais da Alucar no ano
  lucro FLOAT,            -- Lucro anual da Alucar (receita - despesas)
  margem_lucro FLOAT,     -- Margem de lucro anual da Alucar
  evolucao_lucro FLOAT    -- Comparação com o ano anterior da Alucar
);
""",
"""
-- Visão anual da operação da empresa Consigcar
CREATE TABLE IF NOT EXISTS g_fato_financas_anual_consigcar (
  id_data INTEGER NOT NULL, -- Referência a uma data do ano na dim_tempo
  receita_total FLOAT,     -- Receita total gerada pela Consigcar no ano
  despesas_total FLOAT,    -- Despesas totais da Consigcar no ano
  lucro FLOAT,            -- Lucro anual da Consigcar (receita - despesas)
  margem_lucro FLOAT,     -- Margem de lucro anual da Consigcar
  evolucao_lucro FLOAT    -- Comparação com o ano anterior da Consigcar
);
""",
"""
-- DRE simplificada com categorias
CREATE TABLE IF NOT EXISTS g_dre (
  id_data INTEGER NOT NULL PRIMARY KEY, -- Chave primária referenciando a data
  ano INTEGER,                        -- Ano da competência
  mes INTEGER,                        -- Mês da competência
  categoria TEXT,                    -- Categoria contábil da despesa ou receita, ex: Receita Bruta, Impostos, Custo, Despesas Operacionais
  origem TEXT,                      -- Origem da despesa/receita
  tipo TEXT,                        -- Indica se é Receita ou Despesa
  valor FLOAT                       -- Valor financeiro da linha
);
""",
"""
-- PLR - Vendas por vendedor com ranking mensal
CREATE TABLE IF NOT EXISTS g_plr_vendas_vendedor_mensal (
  id_vendedor INTEGER NOT NULL,        -- Referência ao vendedor
  id_data INTEGER NOT NULL,             -- Referência à data representando o mês
  total_vendas INTEGER,                 -- Quantidade de vendas realizadas no mês
  valor_parcelas_total FLOAT,          -- Soma dos valores de parcelas (não o total das vendas)
  ranking INTEGER,                    -- Posição no ranking de vendas mensal, baseado em qtd e valor das parcelas
  PRIMARY KEY (id_vendedor, id_data)
);
""",
"""
-- PLR - Vendas por vendedor com ranking nos últimos 10 dias de cada mês
CREATE TABLE IF NOT EXISTS g_plr_vendas_ultimos_10_dias (
  id_data INTEGER NOT NULL,             -- Data de referência, geralmente o último dia do mês
  id_vendedor INTEGER NOT NULL,         -- Referência ao vendedor na dim_vendedor
  total_vendas INTEGER,                 -- Número total de vendas feitas pelo vendedor nos últimos 10 dias do mês
  valor_total FLOAT,                   -- Soma das parcelas das vendas feitas pelo vendedor nesse período
  ranking INTEGER,                    -- Posição do vendedor no ranking dos últimos 10 dias do mês, considerando total_vendas e valor_total
  PRIMARY KEY (id_vendedor, id_data)
);
"""
]

# Executa cada comando separadamente
with engine_gold.begin() as conn:
    for stmt in create_scripts:
        conn.execute(text(stmt))

In [12]:
# Determinação de tipos inteiros para o uso do Power BI
# Defina os dtypes para cada tabela
dtype_financas = {
    'id_data': types.INTEGER(),
    'mes': types.INTEGER(),
    'ano': types.INTEGER(),
    'ranking': types.INTEGER(),
    'total_vendas': types.INTEGER()
}

dtype_plr = {
    'id_data': types.INTEGER(),
    'id_vendedor': types.INTEGER(),
    'total_vendas': types.INTEGER(),
    'ranking': types.INTEGER()
}

# Transformação e preenchimento das colunas

In [ ]:
# g_fato_financas_mensal_alucar
# Tabela contendo dados de financas da empresa Alucar

# Lê apenas meses e anos distintos da tabela s_dim_tempo da camada silver
df_dim_tempo = pd.read_sql_query("SELECT DISTINCT mes, ano FROM s_dim_tempo", engine_silver)

# Gera o primeiro dia de cada mês em formato YYYYMMDD
df_primeiro_dia_mes = df_dim_tempo.apply(lambda x: int(f"{x['ano']}{x['mes']:02d}01"), axis=1).to_frame('id_data')

# Cria o DataFrame para g_fato_financas_mensal com apenas id_data
df_fato_financas_mensal_alucar = pd.DataFrame({
    'id_data': df_primeiro_dia_mes['id_data']
})

# Ordena o DataFrame por id_data
df_fato_financas_mensal_alucar = df_fato_financas_mensal_alucar.sort_values('id_data')

# Calcula a receita total por mês a partir das vendas da Alucar
df_receita = pd.read_sql_query("""
    SELECT 
        (CAST(CAST(dt.ano AS TEXT) || substr('0' || CAST(dt.mes AS TEXT), -2) || '01' AS BIGINT)) as id_data,
        SUM(fva.valor_venda) as receita_total
    FROM fato_vendas_alucar fva
    JOIN s_dim_tempo dt ON fva.id_data = dt.id_data 
    GROUP BY dt.ano, dt.mes
""", engine_silver)

# Faz o merge com o DataFrame principal
df_fato_financas_mensal_alucar = df_fato_financas_mensal_alucar.merge(
    df_receita,
    on='id_data',
    how='left'
)

# Calcula o total de despesas por mês para Alucar
df_despesas = pd.read_sql_query("""
    SELECT 
        (CAST(CAST(dt.ano AS TEXT) || substr('0' || CAST(dt.mes AS TEXT), -2) || '01' AS BIGINT)) as id_data,
        SUM(fd.valor) as despesas_total
    FROM fato_despesas fd
    JOIN s_dim_tempo dt ON fd.id_data = dt.id_data
    WHERE fd.origem = 'Alucar'
    GROUP BY dt.ano, dt.mes
""", engine_silver)

# Faz o merge com o DataFrame principal
df_fato_financas_mensal_alucar = df_fato_financas_mensal_alucar.merge(
    df_despesas,
    on='id_data',
    how='left'
)

# Calcula o lucro (receita - despesas)
df_fato_financas_mensal_alucar['lucro'] = df_fato_financas_mensal_alucar['receita_total'] - df_fato_financas_mensal_alucar['despesas_total']

# Calcula a margem de lucro (lucro / receita)
df_fato_financas_mensal_alucar['margem_lucro'] = df_fato_financas_mensal_alucar['lucro'] / df_fato_financas_mensal_alucar['receita_total']

# Calcula a evolução do lucro (variação percentual em relação ao mês anterior)
df_fato_financas_mensal_alucar['evolucao_lucro'] = df_fato_financas_mensal_alucar['lucro'].pct_change(fill_method=None)

# Preenche valores nulos com 0
df_fato_financas_mensal_alucar = df_fato_financas_mensal_alucar.fillna(0)

# Insere os dados na tabela g_fato_financas_mensal_alucar
df_fato_financas_mensal_alucar.to_sql(
    'g_fato_financas_mensal_alucar', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_financas
)


In [ ]:
# g_fato_financas_mensal_consigcar
# # Cria DataFrame base com todas as datas
df_fato_financas_mensal_consigcar = pd.read_sql_query("""
    WITH max_data AS (
        SELECT MAX(id_data) as max_id_data 
        FROM dim_pagamentos_programados
    )
    SELECT DISTINCT
        (CAST(CAST(ano AS TEXT) || substr('0' || CAST(mes AS TEXT), -2) || '01' AS BIGINT)) as id_data
    FROM s_dim_tempo, max_data
    WHERE id_data <= max_data.max_id_data
    ORDER BY id_data
""", engine_silver)

# Calcula a receita total por mês para Consigcar
df_receita = pd.read_sql_query("""
    SELECT 
        (CAST(CAST(dt.ano AS TEXT) || substr('0' || CAST(dt.mes AS TEXT), -2) || '01' AS BIGINT)) as id_data,
        SUM(fvc.valor_total) as receita_total
    FROM fato_vendas_consigcar fvc
    JOIN s_dim_tempo dt ON fvc.data_primeira_parcela = dt.id_data 
    GROUP BY dt.ano, dt.mes
""", engine_silver)

# Faz o merge com o DataFrame principal
df_fato_financas_mensal_consigcar = df_fato_financas_mensal_consigcar.merge(
    df_receita,
    on='id_data',
    how='left'
)

# Calcula o total de despesas por mês para Consigcar
df_despesas = pd.read_sql_query("""
    SELECT 
        (CAST(CAST(dt.ano AS TEXT) || substr('0' || CAST(dt.mes AS TEXT), -2) || '01' AS BIGINT)) as id_data,
        SUM(fd.valor) as despesas_total
    FROM fato_despesas fd
    JOIN s_dim_tempo dt ON fd.id_data = dt.id_data
    WHERE fd.origem = 'Consigcar'
    GROUP BY dt.ano, dt.mes
""", engine_silver)

# Faz o merge com o DataFrame principal
df_fato_financas_mensal_consigcar = df_fato_financas_mensal_consigcar.merge(
    df_despesas,
    on='id_data',
    how='left'
)

# Calcula o lucro (receita - despesas)
df_fato_financas_mensal_consigcar['lucro'] = df_fato_financas_mensal_consigcar['receita_total'] - df_fato_financas_mensal_consigcar['despesas_total']

# Calcula a margem de lucro (lucro / receita)
df_fato_financas_mensal_consigcar['margem_lucro'] = df_fato_financas_mensal_consigcar['lucro'] / df_fato_financas_mensal_consigcar['receita_total']

# Calcula a evolução do lucro (variação percentual em relação ao mês anterior)
df_fato_financas_mensal_consigcar['evolucao_lucro'] = df_fato_financas_mensal_consigcar['lucro'].pct_change(fill_method=None)

# Preenche valores nuliros com 0
df_fato_financas_mensal_consigcar = df_fato_financas_mensal_consigcar.fillna(0)

# Insere os dados na tabela g_fato_financas_mensal_consigcar
df_fato_financas_mensal_consigcar.to_sql('g_fato_financas_mensal_consigcar', engine_gold, if_exists='replace', index=False)

# Calcula o total de parcelamentos a receber por mês para Consigcar a partir de dim_pagamentos_programados
df_parcelamentos = pd.read_sql_query("""
    SELECT 
        (CAST(CAST(dt.ano AS TEXT) || substr('0' || CAST(dt.mes AS TEXT), -2) || '01' AS BIGINT)) as id_data,
        SUM(dpp.valor) as parcelamentos_receber
    FROM dim_pagamentos_programados dpp
    JOIN s_dim_tempo dt ON dpp.id_data = dt.id_data
    GROUP BY dt.ano, dt.mes
""", engine_silver)

# Faz o merge com o DataFrame principal
df_fato_financas_mensal_consigcar = df_fato_financas_mensal_consigcar.merge(
    df_parcelamentos,
    on='id_data',
    how='left'
)

# Calcula o total faturado pelo PagSeguro por mês
df_faturamento_pagseguro = pd.read_sql_query("""
    SELECT 
        (CAST(CAST(dt.ano AS TEXT) || substr('0' || CAST(dt.mes AS TEXT), -2) || '01' AS BIGINT)) as id_data,
        SUM(CAST(ffp.valor_faturado AS FLOAT)) as faturamento_pagseguro
    FROM fato_faturamento_pagseguro ffp
    JOIN s_dim_tempo dt ON ffp.id_data = dt.id_data
    GROUP BY dt.ano, dt.mes
""", engine_silver)

# Faz o merge com o DataFrame principal
df_fato_financas_mensal_consigcar = df_fato_financas_mensal_consigcar.merge(
    df_faturamento_pagseguro,
    on='id_data',
    how='left'
)

# Preenche valores nulos com 0
df_fato_financas_mensal_consigcar['faturamento_pagseguro'] = df_fato_financas_mensal_consigcar['faturamento_pagseguro'].fillna(0)

# Insere os dados na tabela g_fato_financas_mensal_consigcar
df_fato_financas_mensal_consigcar.to_sql(
    'g_fato_financas_mensal_consigcar', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_financas
)


In [ ]:
# g_fato_financas_mensal
# Lê as tabelas de finanças mensais do Consigcar e Alucar
# Obs.: Essa tabela usa a "Receita" da empresa Consigcar a partir da soma das vendas do mês. 
# Na tabela original, o valor utilizado é o do PagSeguro. Isso precisará ser ajustado caso seja solicitado. 
df_consigcar = pd.read_sql_table('g_fato_financas_mensal_consigcar', engine_gold)
df_alucar = pd.read_sql_table('g_fato_financas_mensal_alucar', engine_gold)

# Faz o merge das duas tabelas usando outer join para pegar todos os meses
df_fato_financas_mensal = pd.merge(
    df_consigcar,
    df_alucar,
    on='id_data',
    how='outer',
    suffixes=('_consigcar', '_alucar')
)

# Preenche valores nulos com 0 para fazer as somas
df_fato_financas_mensal = df_fato_financas_mensal.fillna(0)

# Soma os valores das duas empresas
df_fato_financas_mensal['receita_total'] = df_fato_financas_mensal['receita_total_consigcar'] + df_fato_financas_mensal['receita_total_alucar']
df_fato_financas_mensal['despesas_total'] = df_fato_financas_mensal['despesas_total_consigcar'] + df_fato_financas_mensal['despesas_total_alucar']
df_fato_financas_mensal['lucro'] = df_fato_financas_mensal['lucro_consigcar'] + df_fato_financas_mensal['lucro_alucar']

# Calcula a margem de lucro total
df_fato_financas_mensal['margem_lucro'] = (df_fato_financas_mensal['lucro'] / df_fato_financas_mensal['receita_total']) * 100

# Calcula a evolução do lucro
df_fato_financas_mensal = df_fato_financas_mensal.sort_values('id_data')
df_fato_financas_mensal['evolucao_lucro'] = df_fato_financas_mensal['lucro'].pct_change() * 100

# Seleciona apenas as colunas necessárias
df_fato_financas_mensal = df_fato_financas_mensal[[
    'id_data',
    'receita_total',
    'despesas_total',
    'lucro',
    'margem_lucro',
    'evolucao_lucro'
]]

# Insere os dados na tabela g_fato_financas_mensal
df_fato_financas_mensal.to_sql(
    'g_fato_financas_mensal', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_financas
)

# Imprime o DataFrame para verificação apenas de 2025
print(df_fato_financas_mensal[df_fato_financas_mensal['id_data'].astype(str).str.startswith('2025')])



In [ ]:
# g_fato_financas_anual_alucar
# Lê a tabela mensal da Alucar que já foi criada
df_alucar = pd.read_sql_table('g_fato_financas_mensal_alucar', engine_gold)

# Converte id_data para string e pega só o ano (primeiros 4 dígitos)
df_alucar['ano'] = df_alucar['id_data'].astype(str).str[:4]

# Agrupa por ano e soma os valores
df_alucar_anual = df_alucar.groupby('ano').agg({
    'receita_total': 'sum',
    'despesas_total': 'sum',
    'lucro': 'sum'
}).reset_index()

# Converte ano de volta para id_data (ano + "0101")
df_alucar_anual['id_data'] = df_alucar_anual['ano'] + '0101'

# Remove a coluna ano
df_alucar_anual = df_alucar_anual.drop('ano', axis=1)

# Calcula a margem de lucro
df_alucar_anual['margem_lucro'] = (df_alucar_anual['lucro'] / df_alucar_anual['receita_total']) * 100

# Calcula a evolução do lucro
df_alucar_anual = df_alucar_anual.sort_values('id_data')
df_alucar_anual['evolucao_lucro'] = df_alucar_anual['lucro'].pct_change() * 100

# Seleciona e ordena as colunas necessárias
df_alucar_anual = df_alucar_anual[[
    'id_data',
    'receita_total',
    'despesas_total',
    'lucro',
    'margem_lucro',
    'evolucao_lucro'
]]

# Converte id_data para BIGINT
df_alucar_anual['id_data'] = df_alucar_anual['id_data'].astype('int64')

# Insere os dados na tabela g_fato_financas_anual_alucar
df_alucar_anual.to_sql(
    'g_fato_financas_anual_alucar', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_financas
)

# Imprime o DataFrame para verificação
print("\nDados anuais da Alucar:")
print(df_alucar_anual)



In [ ]:
# g_fato_financas_mensal_consigcar

# Lê a tabela g_fato_financas_mensal_consigcar
df_consigcar = pd.read_sql_table('g_fato_financas_mensal_consigcar', engine_gold)

# Converte id_data para string e pega só o ano (primeiros 4 dígitos)
df_consigcar['ano'] = df_consigcar['id_data'].astype(str).str[:4]

# Agrupa por ano e soma os valores
df_consigcar_anual = df_consigcar.groupby('ano').agg({
    'receita_total': 'sum',
    'despesas_total': 'sum',
    'lucro': 'sum'
}).reset_index()

# Converte ano de volta para id_data (ano + "0101")
df_consigcar_anual['id_data'] = df_consigcar_anual['ano'] + '0101'

# Remove a coluna ano
df_consigcar_anual = df_consigcar_anual.drop('ano', axis=1)

# Calcula a margem de lucro
df_consigcar_anual['margem_lucro'] = (df_consigcar_anual['lucro'] / df_consigcar_anual['receita_total']) * 100

# Calcula a evolução do lucro
df_consigcar_anual = df_consigcar_anual.sort_values('id_data')
df_consigcar_anual['evolucao_lucro'] = df_consigcar_anual['lucro'].pct_change() * 100

# Seleciona e ordena as colunas necessárias
df_consigcar_anual = df_consigcar_anual[[
    'id_data',
    'receita_total',
    'despesas_total',
    'lucro',
    'margem_lucro',
    'evolucao_lucro'
]]

# Converte id_data para BIGINT
df_consigcar_anual['id_data'] = df_consigcar_anual['id_data'].astype('int64')

# Insere os dados na tabela g_fato_financas_anual_consigcar
df_consigcar_anual.to_sql(
    'g_fato_financas_anual_consigcar', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_financas
)

# Imprime o DataFrame para verificação
print("\nDados anuais da Consigcar:")
print(df_consigcar_anual)


In [11]:
# g_dre
# Preciso ver como fazer essa tabela

In [ ]:
# g_plr_vendas_vendedor_mensal
# Apaga todos os registros existentes da tabela g_plr_vendas_vendedor_mensal
with engine_gold.connect() as conn:
    conn.execute(text("DELETE FROM g_plr_vendas_vendedor_mensal"))

# Lê os dados da tabela fato_vendas_diaria_vendedor
df_vendas_diarias = pd.read_sql("""
    SELECT 
        id_vendedor,
        id_data,
        total_vendas,
        valor_total
    FROM fato_vendas_diaria_vendedor
""", engine_silver)

# Extrai mês e ano do id_data para agrupar
df_vendas_diarias['mes_ano'] = df_vendas_diarias['id_data'].astype(str).str[:6]

# Agrupa por vendedor e mês, somando os totais
df_vendas_mensais = df_vendas_diarias.groupby(['id_vendedor', 'mes_ano']).agg({
    'total_vendas': 'sum',
    'valor_total': 'sum'
}).reset_index()

# Converte mes_ano para id_data (adiciona "01" ao final)
df_vendas_mensais['id_data'] = df_vendas_mensais['mes_ano'] + '01'
df_vendas_mensais = df_vendas_mensais.drop('mes_ano', axis=1)

# Calcula o ranking mensal baseado no valor total
df_vendas_mensais['ranking'] = df_vendas_mensais.groupby('id_data')['valor_total'].rank(ascending=False, method='dense').astype(int)

# Renomeia a coluna valor_total para valor_parcelas_total para manter consistência com o modelo
df_vendas_mensais = df_vendas_mensais.rename(columns={'valor_total': 'valor_parcelas_total'})

# Converte id_data para BIGINT
df_vendas_mensais['id_data'] = df_vendas_mensais['id_data'].astype('int64')

# Ordena as colunas conforme o modelo
df_vendas_mensais = df_vendas_mensais[[
    'id_vendedor',
    'id_data',
    'total_vendas',
    'valor_parcelas_total',
    'ranking'
]]

# Insere os dados na tabela g_plr_vendas_vendedor_mensal
df_vendas_mensais.to_sql(
    'g_plr_vendas_vendedor_mensal', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_plr
)

In [ ]:
#g_plr_vendas_vendedor_diario
# Lê os dados da tabela fato_vendas_diaria_vendedor sem transformações
df_vendas_diarias = pd.read_sql("""
    SELECT *
    FROM fato_vendas_diaria_vendedor
""", engine_silver)

# Insere os dados na tabela g_plr_vendas_vendedor_diario mantendo a estrutura original
df_vendas_diarias.to_sql(
    'g_plr_vendas_vendedor_diario',
    engine_gold,
    if_exists='replace',
    index=False
)

In [ ]:
# g_plr_vendas_ultimos_10_dias
# Apaga todos os registros existentes da tabela g_plr_vendas_ultimos_10_dias
with engine_gold.connect() as conn:
    conn.execute(text("DELETE FROM g_plr_vendas_ultimos_10_dias"))

# Lê os dados da tabela fato_vendas_diaria_vendedor
df_vendas_diarias = pd.read_sql("""
    SELECT 
        id_vendedor,
        id_data,
        total_vendas,
        valor_total
    FROM fato_vendas_diaria_vendedor
""", engine_silver)

# Converte id_data para datetime
df_vendas_diarias['data'] = pd.to_datetime(df_vendas_diarias['id_data'].astype(str), format='%Y%m%d')

# Calcula o último dia de cada mês
df_vendas_diarias['ultimo_dia_mes'] = df_vendas_diarias['data'] + pd.offsets.MonthEnd(0)

# Filtra apenas os últimos 10 dias de cada mês
df_vendas_ultimos_10_dias = df_vendas_diarias[df_vendas_diarias['data'] >= df_vendas_diarias['ultimo_dia_mes'] - pd.Timedelta(days=9)]

# Agrupa por vendedor e data, somando os totais
df_vendas_agrupadas = df_vendas_ultimos_10_dias.groupby(['id_vendedor', 'id_data']).agg({
    'total_vendas': 'sum',
    'valor_total': 'sum'
}).reset_index()

# Calcula o ranking diário baseado no valor total
df_vendas_agrupadas['ranking'] = df_vendas_agrupadas.groupby('id_data')['valor_total'].rank(ascending=False, method='dense').astype(int)

# Converte id_data para BIGINT
df_vendas_agrupadas['id_data'] = df_vendas_agrupadas['id_data'].astype('int64')

# Ordena as colunas conforme o modelo
df_vendas_agrupadas = df_vendas_agrupadas[[
    'id_data',
    'id_vendedor',
    'total_vendas',
    'valor_total',
    'ranking'
]]
df_vendas_mensais = df_vendas_mensais[[
    'id_vendedor',
    'id_data',
    'total_vendas',
    'valor_parcelas_total',
    'ranking'
]]

# Insere os dados na tabela g_plr_vendas_ultimos_10_dias
df_vendas_mensais.to_sql(
    'g_plr_vendas_ultimos_10_dias', 
    engine_gold, 
    if_exists='replace', 
    index=False,
    dtype=dtype_plr
)


In [ ]:
# g_vendas_mensal_alucar
# Calcula vendas mensais da Alucar
df_vendas_mensais_alucar = pd.read_sql_table('s_fato_vendas_alucar', engine_silver)

# Converte id_data para datetime
df_vendas_mensais_alucar['data'] = pd.to_datetime(df_vendas_mensais_alucar['id_data'].astype(str), format='%Y%m%d')

# Extrai ano e mês
df_vendas_mensais_alucar['ano_mes'] = df_vendas_mensais_alucar['data'].dt.to_period('M')

# Agrupa por ano_mes e conta as vendas
df_vendas_mensais_alucar = df_vendas_mensais_alucar.groupby('ano_mes').agg(
    total_vendas=('id_venda_alucar', 'count'),
    valor_total=('valor_venda', 'sum')
).reset_index()

# Converte ano_mes para formato YYYYMM
df_vendas_mensais_alucar['id_data'] = df_vendas_mensais_alucar['ano_mes'].dt.strftime('%Y%m').astype(int)

# Seleciona colunas finais
df_vendas_mensais_alucar = df_vendas_mensais_alucar[['id_data', 'total_vendas', 'valor_total']]

# Insere na tabela gold
df_vendas_mensais_alucar.to_sql(
    'g_vendas_mensal_alucar',
    engine_gold,
    if_exists='replace',
    index=False,
    dtype={
        'id_data': 'BIGINT',
        'total_vendas': 'BIGINT',
        'valor_total': 'FLOAT'
    }
)

df_vendas_mensais_alucar.display()

In [ ]:
# g_vendas_mensal_consigcar
# Calcula vendas mensais da Consigcar
df_vendas_mensais_consigcar = pd.read_sql_table('s_fato_vendas_consigcar', engine_silver)

# Converte id_data para datetime usando data_primeira_parcela
df_vendas_mensais_consigcar['data'] = pd.to_datetime(df_vendas_mensais_consigcar['data_primeira_parcela'].astype(str), format='%Y%m%d')

# Extrai ano e mês
df_vendas_mensais_consigcar['ano_mes'] = df_vendas_mensais_consigcar['data'].dt.to_period('M')

# Agrupa por ano_mes e conta as vendas
df_vendas_mensais_consigcar = df_vendas_mensais_consigcar.groupby('ano_mes').agg(
    total_vendas=('id_venda_consigcar', 'count'),
    valor_total=('valor_total', 'sum')
).reset_index()

# Converte ano_mes para formato YYYYMM
df_vendas_mensais_consigcar['id_data'] = df_vendas_mensais_consigcar['ano_mes'].dt.strftime('%Y%m').astype(int)

# Seleciona colunas finais
df_vendas_mensais_consigcar = df_vendas_mensais_consigcar[['id_data', 'total_vendas', 'valor_total']]

# Insere na tabela gold
df_vendas_mensais_consigcar.to_sql(
    'g_vendas_mensal_consigcar',
    engine_gold,
    if_exists='replace',
    index=False,
    dtype={
        'id_data': 'BIGINT',
        'total_vendas': 'BIGINT',
        'valor_total': 'FLOAT'
    }
)

df_vendas_mensais_consigcar.display()

In [14]:
# Transformar o banco de dados em DBML para acompanhamento da modelagem

dbml = sql_to_dbml(engine_gold)
with open("../modelagem/02_gold.dbml", "w", encoding="utf-8") as f:
    f.write(dbml)